In [78]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import dual_annealing
import sys


In [21]:
def create_grid(bounds: np.ndarray = np.array([[1, 10], [1, 10]]),
                 size: np.ndarray = np.array([10,15])) -> np.ndarray:
    """ создание прямоугольной
    неравномерной сетки """
    # a = np.zeros((np.shape(bounds)[0],1),dtype=np.ndarray)
    a = np.array(bounds[:,0],dtype=np.ndarray)
    for i in range(np.shape(bounds)[0]):
        a[i] = np.linspace(bounds[i, 0], bounds[i, 1], size[i])
    return a

In [22]:
asd = create_grid(np.array([[1, 10], [1, 10],[2, 50],[3, 60]]),np.array([10,15,12,13]))

In [23]:
asd

array([array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]),
       array([ 1.        ,  1.64285714,  2.28571429,  2.92857143,  3.57142857,
               4.21428571,  4.85714286,  5.5       ,  6.14285714,  6.78571429,
               7.42857143,  8.07142857,  8.71428571,  9.35714286, 10.        ]),
       array([ 2.        ,  6.36363636, 10.72727273, 15.09090909, 19.45454545,
              23.81818182, 28.18181818, 32.54545455, 36.90909091, 41.27272727,
              45.63636364, 50.        ])                                      ,
       array([ 3.  ,  7.75, 12.5 , 17.25, 22.  , 26.75, 31.5 , 36.25, 41.  ,
              45.75, 50.5 , 55.25, 60.  ])                                  ],
      dtype=object)

In [24]:
create_grid()

array([array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]),
       array([ 1.        ,  1.64285714,  2.28571429,  2.92857143,  3.57142857,
               4.21428571,  4.85714286,  5.5       ,  6.14285714,  6.78571429,
               7.42857143,  8.07142857,  8.71428571,  9.35714286, 10.        ])],
      dtype=object)

In [25]:
asd[2]

array([ 2.        ,  6.36363636, 10.72727273, 15.09090909, 19.45454545,
       23.81818182, 28.18181818, 32.54545455, 36.90909091, 41.27272727,
       45.63636364, 50.        ])

In [26]:
def func2(x: np.ndarray):
    return -np.sin(0.5 * (x[0]**2) + - 0.25 * (x[1]**2) + 3) * np.cos(2*x[0] + 1 - np.exp(x[1]))

In [27]:
def grid_of_funceval(value_grid: np.ndarray,
                      func) -> np.ndarray:
    """ только для пространсва D=2, для любой сетки в том числе и неравномерной"""
    idk = np.max(np.array([np.shape(value_grid[0])[0],np.shape(value_grid[1])[0]]))
    grid = np.full((idk,idk), np.inf)
    for i in range(np.shape(value_grid[0])[0]):
        for k in range(np.shape(value_grid[1])[0]):
            # grid[i,k] = func(value_grid[np.arange(0, np.shape(value_grid)[0]), qq])
            grid[i,k] = func(np.array([(value_grid[0])[i],(value_grid[1])[k]]))
    return grid

In [28]:
asd = create_grid()

In [29]:
np.shape(asd[0])[0]

10

In [30]:
grid = grid_of_funceval(asd, func2)

In [31]:
asd

array([array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]),
       array([ 1.        ,  1.64285714,  2.28571429,  2.92857143,  3.57142857,
               4.21428571,  4.85714286,  5.5       ,  6.14285714,  6.78571429,
               7.42857143,  8.07142857,  8.71428571,  9.35714286, 10.        ])],
      dtype=object)

In [32]:
grid[9,14]

0.0485142035921649

In [33]:
func2([1,1])

0.10392999507424829

In [34]:
grid.shape

(15, 15)

In [35]:
def min_of_function(grid: np.ndarray) -> np.ndarray:
    return [np.where(grid == np.min(grid))[0][0],np.where(grid == np.min(grid))[1][0]],np.min(grid)

In [36]:
min_of_function(grid)

([5, 11], -0.9990248207756316)

In [37]:
np.where(grid == np.min(grid))[0][0]

5

In [39]:
asd = create_grid()

In [40]:
asd

array([array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]),
       array([ 1.        ,  1.64285714,  2.28571429,  2.92857143,  3.57142857,
               4.21428571,  4.85714286,  5.5       ,  6.14285714,  6.78571429,
               7.42857143,  8.07142857,  8.71428571,  9.35714286, 10.        ])],
      dtype=object)

In [42]:
def start_solve(value_grid: np.ndarray) -> np.ndarray:
    a = np.array([])
    index1 = np.array([],dtype=int)
    for i in range(np.shape(value_grid)[0]):
        index = np.random.randint(np.shape(value_grid[i])[0])
        a = np.append(a,(value_grid[i])[index])
        index1 = np.append(index1,index)
    return a, index1

In [45]:
a,a1 = start_solve(asd)

In [46]:
a

array([7.        , 4.21428571])

In [47]:
a1

array([6, 5])

In [48]:
def new_point_boltzman(bounds: np.ndarray,
                        value_grid: np.ndarray,
                        temperature: float,
                        index: np.ndarray) -> np.ndarray:
    std = np.sqrt(temperature) * np.ones(np.shape(bounds)[0])
    xc = np.random.normal(0, 1.0, size =  np.shape(bounds)[0])
    new_solve_index = index + xc * std
    new_solve = np.copy(new_solve_index)
    for i in range(np.shape(bounds)[0]):
        new_solve_index[i] = round(index[i] + xc[i] * std[i] * (np.shape((value_grid[i]))[0] - 1))
        while  new_solve_index[i] < 0 or  new_solve_index[i] > (np.shape((value_grid[i]))[0] - 1):
            xc[i] = np.random.normal(0, 1.0)
            new_solve_index[i] = round(index[i] + xc[i] * std[i] * (np.shape((value_grid[i]))[0] - 1))
    for i in range(np.shape(bounds)[0]):
        new_solve[i] = (value_grid[i])[new_solve_index.astype(int)[i]]
    return new_solve, new_solve_index.astype(int)

In [53]:
new_point_boltzman(np.array([[1, 10], [1, 10]]),asd,10,a1)

(array([7.        , 7.42857143]), array([ 6, 10]))

In [52]:
asd

array([array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]),
       array([ 1.        ,  1.64285714,  2.28571429,  2.92857143,  3.57142857,
               4.21428571,  4.85714286,  5.5       ,  6.14285714,  6.78571429,
               7.42857143,  8.07142857,  8.71428571,  9.35714286, 10.        ])],
      dtype=object)

In [54]:
asd = create_grid(np.array([[1, 10], [1, 10],[2, 50],[3, 60]]),np.array([10,15,12,13]))

In [55]:
asd

array([array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]),
       array([ 1.        ,  1.64285714,  2.28571429,  2.92857143,  3.57142857,
               4.21428571,  4.85714286,  5.5       ,  6.14285714,  6.78571429,
               7.42857143,  8.07142857,  8.71428571,  9.35714286, 10.        ]),
       array([ 2.        ,  6.36363636, 10.72727273, 15.09090909, 19.45454545,
              23.81818182, 28.18181818, 32.54545455, 36.90909091, 41.27272727,
              45.63636364, 50.        ])                                      ,
       array([ 3.  ,  7.75, 12.5 , 17.25, 22.  , 26.75, 31.5 , 36.25, 41.  ,
              45.75, 50.5 , 55.25, 60.  ])                                  ],
      dtype=object)

In [56]:
solve1, index1 = start_solve(asd)

In [57]:
solve1

array([ 5.        ,  7.42857143, 28.18181818, 26.75      ])

In [58]:
index1

array([ 4, 10,  6,  5])

In [60]:
grid_of_values = np.full(shape = np.array([10,15,12,13]), fill_value = np.NAN)

In [63]:
np.shape(grid_of_values)

(10, 15, 12, 13)

In [64]:
grid_of_values[1,2,3,4]

nan

In [65]:
grid_of_values[tuple(index1)]

nan

In [66]:
solve3, index3 = new_point_boltzman(np.array([[1, 10], [1, 10],[2, 50],[3, 60]]), asd, 100,  index1)

In [67]:
solve3

array([ 6.        ,  3.57142857, 15.09090909, 41.        ])

In [68]:
index3

array([5, 4, 3, 8])

In [84]:
def func2(x: np.ndarray):
    return -np.sin(0.5 * (x[0]**2) + - 0.25 * (x[1]**2) + 3) * np.cos(2*x[0] + 1 - np.exp(x[1]))

In [114]:
lw = [9] * 2
up = [10] * 2
bounds=list(zip(lw, up))

In [115]:
ret = dual_annealing(func2,bounds=np.array([[9, 10], [9, 10]]))

In [116]:
bounds

[(9, 10), (9, 10)]

In [117]:
ret

 message: ['Maximum number of iteration reached']
 success: True
  status: 0
     fun: -0.9999999989253762
       x: [ 9.996e+00  9.616e+00]
     nit: 1000
    nfev: 4139
    njev: 46
    nhev: 0

In [118]:
ret.fun

-0.9999999989253762